In [45]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import mediapipe as mp

In [46]:
def draw_landmarks_on_image(rgb_image, detection_result):
  face_landmarks_list = detection_result.face_landmarks[0]
  annotated_image = np.copy(rgb_image)

  # Loop through the detected faces to visualize.
  for idx in range(len(face_landmarks_list)):
    face_landmarks = face_landmarks_list[idx]

    # Draw the face landmarks.
    face_landmarks_proto = face_landmarks.NormalizedLandmarkList()
    face_landmarks_proto.landmark.extend([
      face_landmarks.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in face_landmarks
    ])

    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_TESSELATION,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp.solutions.drawing_styles
        .get_default_face_mesh_tesselation_style())
    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_CONTOURS,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp.solutions.drawing_styles
        .get_default_face_mesh_contours_style())
    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_IRISES,
          landmark_drawing_spec=None,
          connection_drawing_spec=mp.solutions.drawing_styles
          .get_default_face_mesh_iris_connections_style())

  return annotated_image

In [47]:
model_path = "face_landmarker.task"

In [48]:
BaseOptions = mp.tasks.BaseOptions
FaceLandmarker = mp.tasks.vision.FaceLandmarker
FaceLandmarkerOptions = mp.tasks.vision.FaceLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

options = FaceLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.IMAGE)

In [49]:
mp_drawing = mp.solutions.drawing_utils

In [50]:
cap = cv2.VideoCapture(0)
while True:
    ret, image = cap.read()
    
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break

    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=image)   
    detector = FaceLandmarker.create_from_options(options)
    face_landmarker_result = detector.detect(mp_image)
    annotated_image = draw_landmarks_on_image(mp_image, face_landmarker_result)
    
    cv2.imshow('frame', annotated_image)
    
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

W20230905 13:11:34.205341 12774022 face_landmarker_graph.cc:168] Face blendshape model contains CPU only ops. Sets FaceBlendshapesGraph acceleration to Xnnpack.


AttributeError: 'NormalizedLandmark' object has no attribute 'NormalizedLandmarkList'

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [7]:
# Получение ывидео потока с камеры
cap = cv2.VideoCapture(0)
for i in range(1):
    ret, image = cap.read()
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break

    # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(image)
    print(results.__dict__)

    mp_drawing.draw_landmarks(image, results.multi_face_landmarks[0])
    cv2.imshow('frame', image)
    
    if cv2.waitKey(1) == ord('q'):
        break
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

{'__doc__': 'SolutionOutputs(multi_face_landmarks,)', '__slots__': (), '_fields': ('multi_face_landmarks',), '_field_defaults': {}, '_fields_defaults': {}, '__new__': <staticmethod object at 0x16684d3a0>, '_make': <classmethod object at 0x16683b1f0>, '_replace': <function SolutionOutputs._replace at 0x1792d2af0>, '__repr__': <function SolutionOutputs.__repr__ at 0x1792d2ee0>, '_asdict': <function SolutionOutputs._asdict at 0x1792d2d30>, '__getnewargs__': <function SolutionOutputs.__getnewargs__ at 0x179439a60>, 'multi_face_landmarks': [landmark {
  x: 0.5957419
  y: 0.772384
  z: -0.03229069
}
landmark {
  x: 0.60195214
  y: 0.7023346
  z: -0.06348286
}
landmark {
  x: 0.597809
  y: 0.72238296
  z: -0.03321223
}
landmark {
  x: 0.5912356
  y: 0.62787485
  z: -0.049306266
}
landmark {
  x: 0.6028654
  y: 0.6803723
  z: -0.067614175
}
landmark {
  x: 0.6026372
  y: 0.6507138
  z: -0.06313312
}
landmark {
  x: 0.6007401
  y: 0.579868
  z: -0.03212426
}
landmark {
  x: 0.51180243
  y: 0.55

In [2]:
# считывание фотографии
img = cv2.imread("image.jpg")
cv2_imshow(img)

[ WARN:0@0.405] global loadsave.cpp:248 findDecoder imread_('image.jpg'): can't open/read file: check file path/integrity


NameError: name 'cv2_imshow' is not defined